In [4]:
import praw          # Python Reddit API wrapper- pip install if you don't have it
import json
import pprint
import numpy as np
import pandas as pd
import sklearn.feature_extraction.text
import math

In [5]:
# This creates a Reddit instance
reddit = praw.Reddit(client_id = 'kCjbiHmfa6QCHg', 
                     client_secret = 'oE4G_WsYBcQUGyWGndq1tJ5Qnzw', 
                     # Put password below
                     # password = PUT THE PASSWORD HERE 
                     password = "CrainChangChangCrain",
                     user_agent = 'myAgent', 
                     # Put the username below
                     # username = USERNAME HERE
                     username = "BootyMcBootyson"
                    )

In [40]:
# Takes a list of strings
# Returns a list of subreddit objects
def get_subreddits(subreddit_names):
    subreddit_list = []
    for i in subreddit_names:
        sr = reddit.subreddit(i)
        subreddit_list.append(sr)
    return subreddit_list

# Takes a single submission object
# Returns dictionary of k = submission id, v = list of features
def parse_comments(submission):
    comments_dict = {}
    # Get past the "more comment" object to extract all comments
    submission.comments.replace_more(limit=0)
    # Get all comments as flattened list
    comments = submission.comments.list()    
    for comment in comments:
        comments_dict[comment.id] = [str(comment.body), str(comment.subreddit), comment.score, 'comment']
    return comments_dict

# For posts, one approach might be to just take the title of the post
# This is because post bodies ("selftext") may just be links or images
# Takes a list of subreddit objects and extracts comments and posts
# Returns dictionary of k = submission id, v = list of features
def parse_top_posts(subreddits):
    content_dict = {}
    for i in subreddits:
        for submission in i.top(time_filter='all', limit = 10):
            # Add an entry to posts_dict where value is a list of features
            content_dict[submission.id] = [str(submission.title), str(submission.subreddit), submission.score, 'post']
            # Now extract comments
            comments_dict_buff = parse_comments(submission)
            content_dict.update(comments_dict_buff)
    return content_dict

# This is a wrapper method that calls the others
# Takes a list of names of subreddits
# Returns a dataframe of posts and features
def create_reddit_dataset(subreddit_names_list):
    subreddits = get_subreddits(subreddit_names_list)
    content_dict = parse_top_posts(subreddits)
    col_names = ['text', 'subreddit', 'score', 'content_type']
    df = pd.DataFrame.from_dict(content_dict, orient='index', columns=col_names)
    return df

In [41]:
s_names = ['politics']
my_data = create_reddit_dataset(s_names)

In [42]:
my_data.head()

,text,subreddit,score,content_type
9uuhl3,"Kim Davis, clerk who refused to sign marriage ...",politics,101996,post
e973fu0,"\nAs a reminder, this subreddit [is for civil ...",politics,1,comment
e973jxg,Good job Kentucky!,politics,13524,comment
e975nkn,"title should be: ""Lady who refused to do job, ...",politics,24216,comment
e973okz,"From what I remember, this awful bitch owes th...",politics,5735,comment


In [ ]:
# List of subreddits to check in

# Maybe: AskReddit, history, philosophy

my_sub_names = ["news", 
                "politics", 
                "worldnews", 
                "PoliticalHumor", 
                "worldpolitics", 
                "Economics", 
                "PoliticalDiscussion"
                "TheDonald",
                "LateStageCapitalism",
                "Libertarian",
                "SandersForPresident", 
                "conservative",
                "socialism",
                "bluemidterm2018",
                "democrats",
                "anarcho_capitalism",
                "Liberal"
                "progressive"
                "neoliberal"
                ]